# Evento evaluativo 1 - 25022021


## Nombres:

Considere el problema de encontrar un camino en la cuadrícula mostrada en la figura anterior desde la posición $s$ hasta la posición $g$. Una pieza puede moverse en la cuadrícula horizontal o verticalmente, una casilla cada vez. No se puede dar ningún paso en una zona sombreada.

1. Represente el problema como un grafo. Cada cuadro de la figura será un nodo. Cada nodo se debe nombrar como un número exceptuando los nodos $s$ y $g$. Cada arco debe tener un costo unitario y la acción será la dirección, i.e., arriba, izquierda, derecha y abajo. Tenga en cuenta que el grafo es cíclico, esto es, los arcos son de ida y regreso.

2. Encuentre la solución del problema a través de profundidad. ¿Qué cambio debe hacer en el código para que sirva en el caso de grafos cíclicos?. En la solución se debe visualizar el camino, donde cada arco debe mostrar la acción respectiva y el costo total del camino.

3. Encuentre la solución del problema a través de amplitud. En la solución se debe visualizar el camino, donde cada arco debe mostrar la acción respectiva y el costo total del camino.


Nota: recuerde agregar al código todas las librerías, clases, funciones, que sean necesarias para ejecutar el código completamente

In [16]:
class Problema_busqueda():
  '''-nodo inicio
  -funcion de vecinos
  -meta (funcion booleana)
  -heuristica'''
  def nodo_inicio(self):
    'retorna el nodo inicio'
    raise NotImplementedError('nodo_inicio')

  def es_meta(self, nodo):
    'retorna verdadero si nodo es meta'
    raise NotImplementedError('es_meta')

  def heuristica(self,n):
    'retorna la heuristica para el nodo n'
    return 0

  def vecinos(self, nodo):
    'retorna la lista de los arcos de los vecinos del nodo'
    raise NotImplementedError('vecinos')


class Arco():
  '''- nodo entrante
  -nodo saliente
  -costo (no negativo)'''
  def __init__(self, nodo_saliente, nodo_entrante, costo = 1, accion = None):
    assert costo >= 0, ('El costo no puede ser negativo para' + str(nodo_saliente) + ' -> ' + str(nodo_entrante))

    self.nodo_saliente = nodo_saliente
    self.nodo_entrante = nodo_entrante
    self.accion = accion
    self.costo = costo

  def __repr__(self):
    if self.accion:
      return str(self.nodo_saliente)+ ' --' + str(self.accion) + ',' + str(self.costo) + '--> ' + str(self.nodo_entrante)
    else:
      return str(self.nodo_saliente)+ ' --' + str(self.costo) + '--> ' + str(self.nodo_entrante)

class Busqueda_grafo(Problema_busqueda):
  '''- lista o conjunto de nodos
  - lista o conjuntos de arcos
  -nodos inicio
  -lista o conjunto nodos meta
  -un diccionario que mapee caada nodo en un valor heuristico'''
  def __init__(self, nodos, arcos, inicio = None, metas = set(), hmap = {}):
    self.veci = {}
    self.nodos = nodos
    for nodo in nodos:
      self.veci[nodo] = []
    self.arcos = arcos
    for arc in arcos:
      self.veci[arc.nodo_saliente].append(arc)
    self.inicio = inicio
    self.metas = metas
    self.hmap = hmap

  def nodo_inicio(self):
    '''retorna el nodo inicio'''
    return self.inicio

  def es_meta(self, nodo):
    '''retorna verdadero si nodo es una meta'''
    return nodo in self.metas

  def heuristica(self, nodo):
    '''retorna un valor para la heuristica de nodo'''
    if nodo in self.heuristica:
      return self.hmap[nodo]
    else:
      return 0

  def vecinos(self, nodo):
    '''retorna los vecinos de nodo'''
    return self.veci[nodo]

  def __repr__(self):
    salida = ""
    costo_total = 0
    for arc in self.arcos:
      salida += str(arc) + " . "
    return salida


class Camino():

  def __init__(self,inicial, arco = None):
    '''inicial puede ser un nodo o un camino'''
    self.inicial = inicial
    self.arco = arco
    if arco is None:
      self.costo = 0
    else:
      self.costo = inicial.costo + arco.costo

  def nodoSaliente(self):
    if self.arco is None:
      return self.inicial
    else:
      return self.arco.nodo_saliente

  def fin(self):
    '''retorna el nodo final del camino'''
    if self.arco is None:
      return self.inicial
    else:
      return self.arco.nodo_entrante #retorna el nodo entrante del ultimo arco

  def nodos(self):
    '''retorna los nodos del camino de atras hacia adelante'''
    actual = self
    while actual.arco is not None:
      yield actual.arco.nodo_entrante
      actual = actual.inicial
    yield actual.inicial

  def nodos_iniciales(self):
    '''retorna todos los nodos antes del nodo final'''
    if self.arco is not None:
      for nd in self.inicial.nodos(): yield nd

  def __repr__(self):
    if self.arco is None:
      return str(self.inicial)
    elif self.arco.accion:
        return (str(self.inicial) + "\n --" + str(self.arco.accion)+ " --> " + str(self.arco.nodo_entrante))
    else:
      return  (str(self.inicial) + "\n -->" + str(self.arco.nodo_entrante))

class Visualizable():
  '''controla la cantidad de detalles por el nivel max_visual'''
  max_nivel_visual = 1

  def visualizar(self, nivel, *args, **nargs):
    if nivel <= self.max_nivel_visual:
      print(*args, **nargs)

class Buscador(Visualizable):
  '''retorna la solucion para un problema de busqueda'''

  def __init__(self, problema):
    self.problema = problema
    self.inicializar_frontera()
    self.num_expansion = 0
    self.agregar_a_frontera(Camino(problema.nodo_inicio()))
    super().__init__()

  def inicializar_frontera(self):
    self.frontera = []

  def frontera_vacia(self):
    return self.frontera == []

  def agregar_a_frontera(self, camino):
    self.frontera.append(camino)

  def buscar_profundidad(self):
    '''retorna el siguiente camino para el problema del nodo inicio
    al nodo meta'''
    while not self.frontera_vacia():
      camino = self.frontera.pop()
      self.visualizar(2,"Expandiendo", camino)
      self.num_expansion += 1
      if self.problema.es_meta(camino.fin()):      #se encontró la solucion
        self.visualizar(2,self.num_expansion, "caminos expandidos",
                        len(self.frontera), " caminos en la frontera")
        no = ("Costo total del camino: " + str(camino.costo))
        return camino, no
      else:
        vecis = self.problema.vecinos(camino.fin()) #vecinos del ultimo nodo
        self.visualizar(2, "Los vecinos son: ", vecis)
        for arco in reversed(list(vecis)):
          if arco.nodo_entrante != camino.nodoSaliente():
            self.agregar_a_frontera(Camino(camino,arco))
        self.visualizar(2,"Frontera: ",self.frontera)
      self.visualizar(2, "No hay mas soluciones. Se expandieron un total de ",
                      self.num_expansion, " caminos")

  def buscar_amplitud(self):
    while not self.frontera_vacia():
      camino = self.frontera.pop(0)
      self.visualizar(2, "Expandiendo", camino)
      self.num_expansion += 1
      if self.problema.es_meta(camino.fin()):
        self.visualizar(2, self.num_expansion, " caminos se han expandido y quedan ",
                        len(self.frontera), " caminos en la frontera")
        no = ("Costo total del camino: " + str(camino.costo))
        return camino, no
      else:
        vecis = self.problema.vecinos(camino.fin())
        self.visualizar(2, "Los vecinos son: ", vecis)
        for arco in list(vecis):
          self.agregar_a_frontera(Camino(camino,arco))
        self.visualizar(2,"Frontera: ",self.frontera)
      self.visualizar(2, "No hay mas soluciones. Se expandieron un total de ",
                      self.num_expansion, " caminos")

In [17]:
nodos = []
arcos = []
renegados = [27,28,29,30,31,34,43,47]      #Lista con los nodos negros
nd = [8,16,24,32,40,48,56,64]              #Lista con los nodos que no tienen vecinos derecho
ni = [1,9,17,25,33,41,49,57]               #Lista con los nodos que no tienen vecinos izquierdo

nodos = {str(i) for i in range(1,65) if i not in renegados} #comprehension

for i in range(1,65):

  vd = i + 1       #vecino derecho
  vi = i - 1       #vecino izquierdo
  vu = i - 8       #vecino arriba
  va = i + 8       #vecino abajo

  if i not in nd and i not in renegados and vd not in renegados:
    arcos.append(Arco(str(i),str(vd),1,'derecha'))
    hola = 0

  if i not in ni and i not in renegados and vi not in renegados:
    arcos.append(Arco(str(i),str(vi),1,'izquierda'))
    hola = 1

  if vu > 0 and i not in renegados and vu not in renegados:
    arcos.append(Arco(str(i),str(vu),1,'arriba'))


  if va < 65 and i not in renegados and va not in renegados:
    arcos.append(Arco(str(i),str(va),1,'abajo'))


problemaCuadricula = Busqueda_grafo(nodos, arcos, inicio = '44', metas = {'19'})  # 44 es S y 19 es G

-

## Solución punto 2

In [18]:
## Solución punto 3

Buscador(problemaCuadricula).buscar_profundidad()

AttributeError: 'list' object has no attribute 'len'